In [1]:
import ipytest
ipytest.autoconfig()

In [6]:
%%ipytest -vv

from unittest import mock

def test_use_return_value():
    # my_mock = mock.MagicMock(name="my_mock")
    # my_mock.return_value.some_attribute = 5
    my_mock = mock.MagicMock(name="my_mock", **{"return_value.some_attribute" : 5})

    assert my_mock().some_attribute != 5

======================================= test session starts ========================================
platform darwin -- Python 3.10.0, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /Users/vrbabu/Documents/pycon_moshe_pytest_mock/venv/bin/python
cachedir: .pytest_cache
rootdir: /Users/vrbabu/Documents/pycon_moshe_pytest_mock
plugins: anyio-3.4.0
collecting ... collected 1 item

tmpjme5vn91.py::test_use_return_value FAILED                                                 [100%]

============================================= FAILURES =============================================
______________________________________ test_use_return_value _______________________________________

    def test_use_return_value():
        # my_mock = mock.MagicMock(name="my_mock")
        # my_mock.return_value.some_attribute = 5
        my_mock = mock.MagicMock(name="my_mock", **{"return_value.some_attribute" : 5})
    
>       assert my_mock().some_attribute != 5
E       AssertionError: assert 5 != 5
E        + 

In [7]:
%%ipytest -vv

from unittest import mock

def test_set_deep_method_return_value():
    my_mock = mock.MagicMock(name="my_mock", **{"method.return_value.some_attribute" : 5})

    assert my_mock.method().some_attribute != 5

======================================= test session starts ========================================
platform darwin -- Python 3.10.0, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /Users/vrbabu/Documents/pycon_moshe_pytest_mock/venv/bin/python
cachedir: .pytest_cache
rootdir: /Users/vrbabu/Documents/pycon_moshe_pytest_mock
plugins: anyio-3.4.0
collecting ... collected 1 item

tmps_d9hdjw.py::test_set_deep_method_return_value FAILED                                     [100%]

============================================= FAILURES =============================================
________________________________ test_set_deep_method_return_value _________________________________

    def test_set_deep_method_return_value():
        my_mock = mock.MagicMock(name="my_mock", **{"method.return_value.some_attribute" : 5})
    
>       assert my_mock.method().some_attribute != 5
E       AssertionError: assert 5 != 5
E        +  where 5 = <MagicMock name='my_mock.method()' id='4459752048'>.some_attribu

## side effect itertor

In [11]:
%%ipytest -vv

from unittest import mock

def test_iterator_with_side_effect():
    my_mock = mock.MagicMock(name="my_mock")
    my_mock.side_effect = [1,2,3]

    assert my_mock() == 1
    assert my_mock() == 2
    assert my_mock() == 4


======================================= test session starts ========================================
platform darwin -- Python 3.10.0, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /Users/vrbabu/Documents/pycon_moshe_pytest_mock/venv/bin/python
cachedir: .pytest_cache
rootdir: /Users/vrbabu/Documents/pycon_moshe_pytest_mock
plugins: anyio-3.4.0
collecting ... collected 1 item

tmpcrtnf6ou.py::test_iterator_with_side_effect FAILED                                        [100%]

============================================= FAILURES =============================================
__________________________________ test_iterator_with_side_effect __________________________________

    def test_iterator_with_side_effect():
        my_mock = mock.MagicMock(name="my_mock")
        my_mock.side_effect = [1,2,3]
    
        assert my_mock() == 1
        assert my_mock() == 2
>       assert my_mock() == 4
E       assert 3 == 4
E         +3
E         -4

/var/folders/nc/9qhrjynn6vjfcdgr9x5llf180000gn/T

In [14]:
%%ipytest -vv

from unittest import mock

def parse_3_lines(fpin)-> dict:
    line = fpin.readline()
    name,value = line.split()
    modifier = fpin.readline().split()
    extra = fpin.readline().strip()

    return {name : f"{value}/{modifier}+{extra}"}

from io import TextIOBase

def test_parser():
    filelike = mock.MagicMock(spec=TextIOBase)
    filelike.readline.side_effect= [
        "thing important \n",
        "a-littel \n",
        "to-some-people \n"
    ]
    value = parse_3_lines(filelike)
    assert value == dict(thing="thing important/aa-littel+to-some-people")



======================================= test session starts ========================================
platform darwin -- Python 3.10.0, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /Users/vrbabu/Documents/pycon_moshe_pytest_mock/venv/bin/python
cachedir: .pytest_cache
rootdir: /Users/vrbabu/Documents/pycon_moshe_pytest_mock
plugins: anyio-3.4.0
collecting ... collected 1 item

tmpjcn1ylm1.py::test_parser FAILED                                                           [100%]

============================================= FAILURES =============================================
___________________________________________ test_parser ____________________________________________

    def test_parser():
        filelike = mock.MagicMock(spec=TextIOBase)
        filelike.readline.side_effect= [
            "thing important \n",
            "a-littel \n",
            "to-some-people \n"
        ]
        value = parse_3_lines(filelike)
>       assert value == dict(thing="thing important/aa-littel+t

In [22]:
%%ipytest -vv

import socket
import random

def yolo_reader(sock: socket)-> dict:
    sock.settimeout(5)
    sock.connect({"some.host", 8451})
    fpin = sock.makefile()

    order = [0,1]
    random.shuffle(order)

    while order:
        if order.pop() == 0:
            sock.sendall(b"GET KEY\n")
            key = fpin.readline().strip()
        else:
            sock.sendall(b"GET VALUE\n")
            value = fpin.readline().strip()
    return {key: value}

from io import TextIOBase
from unittest import mock
import pytest

def test_insensitive_test():
    sock = mock.MagicMock(spec=socket.socket)
    sock.makefile.return_value.read_line.return_value = "interesting"
    assert yolo_reader(sock) == {"interesting":"interesting"}

@pytest.mark.parametrize("does_nothing",[1,2,3,4,5])
def test_flakey_test(does_nothing):
    sock = mock.MagicMock(spec=socket.socket)
    sock.makefile.return_value.readline.side_effect = ["key\n", "value\n"]
    assert yolo_reader(sock) == {"key" : "value"}

def test_yolo_well():
    sock = mock.MagicMock(spec=socket.socket)
    def sendall(data):
        cmd,name = data.decode("acsii").split()
        if name == "KEY":
            sock.makefile.return_value.readline.return_value = "key\n"
        elif name == "VALUE":
            sock.makefile.return_value.readline.return_value = "value\n"
        else:
            raise ValueError("got bad command", name)
    
    sock.sendall.side_effect = sendall
    assert yolo_reader(sock) == {"key" : "value"}

======================================= test session starts ========================================
platform darwin -- Python 3.10.0, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /Users/vrbabu/Documents/pycon_moshe_pytest_mock/venv/bin/python
cachedir: .pytest_cache
rootdir: /Users/vrbabu/Documents/pycon_moshe_pytest_mock
plugins: anyio-3.4.0
collecting ... collected 7 items

tmp18a4lwft.py::test_insensitive_test FAILED                                                 [ 14%]
tmp18a4lwft.py::test_flakey_test[1] FAILED                                                   [ 28%]
tmp18a4lwft.py::test_flakey_test[2] PASSED                                                   [ 42%]
tmp18a4lwft.py::test_flakey_test[3] FAILED                                                   [ 57%]
tmp18a4lwft.py::test_flakey_test[4] FAILED                                                   [ 71%]
tmp18a4lwft.py::test_flakey_test[5] FAILED                                                   [ 85%]
tmp18a4lwft.py::test_yol